In [ ]:
import os
import numpy as np

# Path to the uploaded zip file
# extract_folder_path = r'H:\Magdalena\photometry\fp_recordings\ACh_sensory\sham/'
# output_folder_path = r'H:\Magdalena\photometry\fp_recordings\ACh_sensory\sham_zscored/'

# r'H:\Magdalena\photometry\fp_recordings\ACh_sensory\SNI/new_SNI/'
# r'H:\Magdalena\photometry\fp_recordings\ACh_sensory\sham/new_sham/'

# extract_folder_path = r'H:\Magdalena\photometry\fp_recordings\NA_sensory\new batch\npy_files/'
# output_folder_path = r'H:\Magdalena\photometry\fp_recordings\NA_sensory\new batch\npy_files\new_batch_zscored/'

# extract_folder_path = r'H:\Magdalena\photometry\fp_recordings\NA_sensory\new batch\salient/'
# output_folder_path = r'H:\Magdalena\photometry\fp_recordings\NA_sensory\new batch\salient\zscored/'

extract_folder_path = r'H:\Magdalena\photometry\fp_recordings\NA_sensory\combined old and new batched npy files\sham/'
output_folder_path = r'H:\Magdalena\photometry\fp_recordings\NA_sensory\combined old and new batched npy files\sham\zscored/'

# Function to z-score the data based on the baseline (first 5 seconds)
def z_score_trace(trace, baseline_start, baseline_end):
    baseline = trace[baseline_start:baseline_end]
    baseline_mean = np.mean(baseline)
    baseline_std = np.std(baseline)
    return (trace - baseline_mean) / baseline_std

# Function to z-score trials
def z_score_trials(data_per_animal, trial_types, baseline_duration=5, sampling_rate=100):
    z_scored_trials = {}
    baseline_end = int(baseline_duration * sampling_rate)
    
    for animal, trials in data_per_animal.items():
        animal_trials = {}
        for trial in trial_types:
            if trial in trials:
                # Apply z-scoring to each trial
                z_scored_data = z_score_trace(trials[trial], 0, baseline_end)
                animal_trials[trial] = z_scored_data
        z_scored_trials[animal] = animal_trials
    return z_scored_trials

# Process each npy file in all subfolders
for root, dirs, files in os.walk(extract_folder_path):
    for filename in files:
        if filename.endswith('.npy'):
            # Load the data
            file_path = os.path.join(root, filename)
            data = np.load(file_path, allow_pickle=True).item()
            
            # Get all trial types from the data
            trial_types = list(data.keys())
            
            # Create a dictionary with the original data structure
            input_data = {filename: data}
            
            # Z-score the trials
            z_scored_data = z_score_trials(input_data, trial_types)
            
            # Extract the z-scored data (removing the filename key)
            z_scored_dict = z_scored_data[filename]
            
            # Create the corresponding output directory structure
            relative_path = os.path.relpath(root, extract_folder_path)
            output_dir = os.path.join(output_folder_path, relative_path)
            os.makedirs(output_dir, exist_ok=True)
            
            # Save the z-scored data
            output_filename = f'z_scored_{filename}'
            output_path = os.path.join(output_dir, output_filename)
            np.save(output_path, z_scored_dict)

print("Z-scoring completed. Files saved with 'z_scored_' prefix in the corresponding subfolders under 'grabda_zscored'.")

Z-scoring completed. Files saved with 'z_scored_' prefix in the corresponding subfolders under 'grabda_zscored'.
